In [1]:
import glob
import os
import subprocess
import numpy as np
import pandas as pd


%load_ext autoreload
%autoreload 1
%aimport bb_behavior
%aimport bb_behavior.plot
%aimport bb_behavior.tracking
%aimport bb_behavior.tracking.pipeline

import bb_behavior
import bb_behavior.plot
import bb_behavior.tracking
import bb_behavior.tracking.pipeline

from bb_behavior.tracking.pipeline import get_default_pipeline
from tqdm import tqdm_notebook # progress bar
import math
from bb_tracking.data.constants import DETKEY
#from bb_tracking.tracking import score_id_sim_v
from bb_tracking.tracking import distance_orientations_v, distance_positions_v

from bb_behavior.tracking.pipeline import detect_markers_in_video
from bb_behavior.tracking.pipeline import track_detections_dataframe
from bb_behavior.tracking.pipeline import display_tracking_results

Using Theano backend.


In [2]:
import time
import os

def filename_to_datestring(filname):
    """
    filename can be path
    """
    return os.path.split(filname)[-1].split('.')[0].split('_')[-1]


def string_to_timestamp(datestring):
    """ 
    params
        string: format 2018-08-19-01-08-13
    output
        unix timestamp (float)
    """
    
    return time.mktime(time.strptime(datestring, "%Y-%m-%d-%H-%M-%S"))

def timestamp_to_string(timestamp):
    return time.strftime("%Y-%m-%d-%H-%M-%S", time.localtime(timestamp))    

In [3]:
datestring = filename_to_datestring('e00_2018-08-19-01-08-13.h264')
timestamp = string_to_timestamp(datestring)
string =  timestamp_to_string(timestamp)

print(datestring)
print(timestamp)
print(string)

2018-08-19-01-08-13
1534633693.0
2018-08-19-01-08-13


In [4]:
default_pipeline = get_default_pipeline(localizer_threshold="0.50")
config = dict(tag_pixel_diameter=50,
              n_frames=None,
              confidence_filter_detections=0.08,
              confidence_filter_tracks=0.20,
              coordinate_scale=1.0,
              start_time=None,
              fps=10.0,
              cam_id=0,
              left_leaving_area = 0.30, # Prozente vom Bildschirmrand, zB. bei 1000px und 0.15 -> 0-150px
              right_leaving_area = 0.30,
              px_x_resolution_vid = 1944)
# Hier passiert das eigentliche Tracken und speichern der Ergebnisse:
num_processed_videos = 0
video_data = dict()
frame_info = None
detections = None
videos_w_no_detection = ""

paths = glob.glob(os.path.join('../videos/videos_tags/', '*.h264'))[:5]
for path in paths:
    start_time = config["start_time"]
    cam_id = config["cam_id"]
    try:
        num_processed_videos += 1

        frame_info, detections = detect_markers_in_video(path,
                                                      decoder_pipeline=default_pipeline,#pipeline=pipelines(),
                                                     tag_pixel_diameter=config["tag_pixel_diameter"],
                                                      n_frames=config["n_frames"],
                                                      fps=config["fps"],
                                                     progress="tqdm_notebook"
                                            )
        tracks = track_detections_dataframe(detections,
                                            tracker="../tracker.det_score_fun.frag_score_fun.dill",
                                            confidence_filter_detections=config["confidence_filter_detections"],
                                           confidence_filter_tracks=config["confidence_filter_tracks"],
                                            coordinate_scale=config["coordinate_scale"]
                                           )
        date_string = filename_to_datestring(path)
        tracks['video'] = date_string
        video_data[path] = (frame_info, detections, tracks)
                    
    except ValueError as err: #tritt auf, wenn Video leer ist. In diesem Fall: überspringe video
        try:
            videos_w_no_detection = videos_w_no_detection + path + "\n"
            paths.remove(path)
            # wir arbeiten später nochmal mit paths, daher müssen das leere löschen, weil sonst
            # in video_data kein zugehöriger Value zu Key = file zu finden ist.
        except KeyError as err:
            continue
    except Exception as err:
        print(err)
        raise
    # only first vid: break


/home/tscho/.local/lib/python3.6/site-packages/keras/backend/theano_backend.py:1032: UserWarning: DEPRECATION: the 'ds' parameter is not going to exist anymore as it is going to be replaced by the parameter 'ws'.
  mode='max')
/home/tscho/.local/lib/python3.6/site-packages/keras/backend/theano_backend.py:1032: UserWarning: DEPRECATION: the 'st' parameter is not going to exist anymore as it is going to be replaced by the parameter 'stride'.
  mode='max')
/home/tscho/.local/lib/python3.6/site-packages/keras/backend/theano_backend.py:1032: UserWarning: DEPRECATION: the 'padding' parameter is not going to exist anymore as it is going to be replaced by the parameter 'pad'.
  mode='max')
Widget Javascript not detected.  It may not be installed or enabled properly.


/home/tscho/anaconda3/lib/python3.6/site-packages/sklearn/base.py:312: UserWarning: Trying to unpickle estimator StandardScaler from version 0.20.0 when using version 0.19.0. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/home/tscho/anaconda3/lib/python3.6/site-packages/sklearn/base.py:312: UserWarning: Trying to unpickle estimator LinearSVC from version 0.20.0 when using version 0.19.0. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/home/tscho/anaconda3/lib/python3.6/site-packages/sklearn/base.py:312: UserWarning: Trying to unpickle estimator Pipeline from version 0.20.0 when using version 0.19.0. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/home/tscho/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a fut

/home/tscho/anaconda3/lib/python3.6/site-packages/sklearn/base.py:312: UserWarning: Trying to unpickle estimator StandardScaler from version 0.20.0 when using version 0.19.0. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/home/tscho/anaconda3/lib/python3.6/site-packages/sklearn/base.py:312: UserWarning: Trying to unpickle estimator LinearSVC from version 0.20.0 when using version 0.19.0. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/home/tscho/anaconda3/lib/python3.6/site-packages/sklearn/base.py:312: UserWarning: Trying to unpickle estimator Pipeline from version 0.20.0 when using version 0.19.0. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/home/tscho/anaconda3/lib/python3.6/site-packages/sklearn/base.py:312: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.20.0 when using version 0.19.0. This might lead to breaking code 

Widget Javascript not detected.  It may not be installed or enabled properly.


/home/tscho/anaconda3/lib/python3.6/site-packages/sklearn/base.py:312: UserWarning: Trying to unpickle estimator StandardScaler from version 0.20.0 when using version 0.19.0. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/home/tscho/anaconda3/lib/python3.6/site-packages/sklearn/base.py:312: UserWarning: Trying to unpickle estimator LinearSVC from version 0.20.0 when using version 0.19.0. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/home/tscho/anaconda3/lib/python3.6/site-packages/sklearn/base.py:312: UserWarning: Trying to unpickle estimator Pipeline from version 0.20.0 when using version 0.19.0. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/home/tscho/anaconda3/lib/python3.6/site-packages/sklearn/base.py:312: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.20.0 when using version 0.19.0. This might lead to breaking code 

In [12]:
videos_w_no_detection

'../videos/videos_tags/e00_2018-08-19-01-08-54.h264\n'

In [13]:
paths

['../videos/videos_tags/e00_2018-08-19-01-08-13.h264',
 '../videos/videos_tags/e00_2018-08-19-01-08-33.h264',
 '../videos/videos_tags/e00_2018-08-19-02-20-55.h264',
 '../videos/videos_tags/e00_2018-08-19-02-21-16.h264']

In [14]:
video_data.keys()

dict_keys(['../videos/videos_tags/e00_2018-08-19-01-08-13.h264', '../videos/videos_tags/e00_2018-08-19-01-08-33.h264', '../videos/videos_tags/e00_2018-08-19-02-21-16.h264'])

In [15]:
tracks = [video_data[paths[x]][2] for x in range(len(paths))]
tracks = pd.concat(tracks,ignore_index=True)
tracks = tracks.drop(columns=["localizerSaliency", "beeID", "camID", "frameIdx"])

KeyError: '../videos/videos_tags/e00_2018-08-19-02-20-55.h264'

In [ ]:
# hyperparameters

tag_pixel_diameter = 50
n_frames = None
fps = 10.0
progress = "tqdm_notebook"
tracker = "../tracker.det_score_fun.frag_score_fun.dill"
confidence_filter_detections = 0.08
confidence_filter_tracks = 0.2
coordinate_scale = 1.0
localizer_threshold="0.55"

#1944

default_pipeline = get_default_pipeline(localizer_threshold=localizer_threshold)

track_dfs = []

counter = 0
for path in glob.glob(os.path.join('../videos/videos_tags/', '*.h264'))[:4]:  

    print(counter, ": ", path)
    
    try:

        # get the detections
        frame_info, detections = detect_markers_in_video(path,
                                                decoder_pipeline=default_pipeline,
                                                tag_pixel_diameter=tag_pixel_diameter,
                                                n_frames=n_frames,
                                                fps=fps,
                                                progress=progress)

        print(len(detections))
        # get the tracks                                           
        tracks = track_detections_dataframe(detections,
                                            tracker=tracker,
                                            confidence_filter_detections=confidence_filter_detections,
                                            confidence_filter_tracks=confidence_filter_tracks,
                                            coordinate_scale=coordinate_scale)
        t = tracks.copy()
        # get date from filename
        date_string = filename_to_datestring(path)
        timestamp = string_to_timestamp(date_string)

        #  transform tracks df to : bee_id, [list of positions (x,y)], [list of timestamps], timestamp_of video
        grouped = tracks.groupby(['bee_id','track_id'],  as_index=False)['bee_id','xpos','ypos','timestamp']
        df = grouped.aggregate(lambda x: list(x))
        df = df.drop('track_id', 1)

        df['timestamp_video'] = timestamp
        df = df.rename(columns={'timestamp': 'timestamps'})

        # convert bee_id list to single value
        df['bee_id'] = df['bee_id'].apply(lambda x: x[0])

        # only appen if note detections?
        track_dfs.append(df)
    
    except ValueError as err: #tritt auf, wenn Video leer ist. In diesem Fall: überspringe video
        print(err)
    except Exception as err:
        print(err)
        raise
    
# bind dataframes together
tracks = pd.concat(track_dfs,ignore_index=True)

In [ ]:
# merge tracks of same bee where start and end timestamps are close together
# assume there can not be overlapping tracks
# 1. sort: bee_id, start_time

# calculate start time of track by adding timestamp of track (seconds since start of video)
# to timestamp of video (date)
tracks['start_time'] = tracks['timestamp_video'] + tracks['timestamps'].apply(lambda x: x[0])
tracks['end_time'] = tracks['timestamp_video'] + tracks['timestamps'].apply(lambda x: x[-1])

In [ ]:
tracks3 = tracks.copy()

In [ ]:
tracks['timestamps'].apply(lambda x: x[-1]) - tracks['timestamps'].apply(lambda x: x[0])

In [ ]:
tracks['end_time'] - tracks['start_time']

In [ ]:
t

In [ ]:
tracks

In [ ]:
tracks = tracks3.copy()
tracks = tracks.sort_values(['bee_id', 'start_time'])

index = 0
tracks2 = tracks.copy()
while(True):
    row = tracks.iloc[index]
    next_row = tracks.iloc[index+1]
    
    # if the tracks are from same bee and the start time of next row is closer then 10s -> merge the rows
    # merge rows means, next_row is deleted
    if (row['bee_id'] == next_row['bee_id']) and ((next_row['start_time'] - row['end_time']) < 10):
        print(next_row['start_time'] - row['end_time'], row['bee_id'], next_row['bee_id'])

        # update the timestamps of nextrow
        t = next_row['start_time'] - row['start_time']
        timestamps = list(np.array(next_row['timestamps']) + t)
        
        tracks.at[row.name,'xpos'] = row['xpos']+next_row['xpos']
        tracks.at[row.name,'ypos'] = row['ypos']+next_row['ypos']
        tracks.at[row.name,'timestamps'] = row['timestamps']+timestamps
        tracks.at[row.name,'end_time'] = next_row['end_time']
        
        tracks.drop(tracks.index[index+1], inplace=True)

    else:
        index += 1
    
    if index == len(tracks) - 1:
        break

In [16]:
tracks.iloc[1]["timestamps"]

KeyError: 'timestamps'

In [ ]:
tracks = tracks3.copy()
tracks = tracks.sort_values(['bee_id', 'start_time'])
for index, rows in tracks.iterrows():
    row = tracks.iloc[index]
    next_row = tracks.iloc[index+1]
    print(next_row['start_time'] - row['end_time'], row['bee_id'], next_row['bee_id'])

In [ ]:
tracks3

In [ ]:
tracks2

In [ ]:
tracks2

In [ ]:
list(np.array([1,2,3,4,5]))

In [ ]:
tracks.iloc[0]

In [ ]:
# hyperparameters

#e00_2018-08-19-02-20-55

tag_pixel_diameter = 50
n_frames = None
fps = 10.0
progress = "tqdm_notebook"
tracker = "../tracker.det_score_fun.frag_score_fun.dill"
confidence_filter_detections = 0.08 # 0.08
confidence_filter_tracks = 0.2 #.2
coordinate_scale = 1.0
localizer_threshold="0.50" # 50

#1944

default_pipeline = get_default_pipeline(localizer_threshold=localizer_threshold)

track_dfs = []
counter = 0
for path in ['../videos/videos_tags/e00_2018-08-19-01-08-13.h264']:  

    print(counter, ": ", path)
    
    try:

        # get the detections
        frame_info, detections = detect_markers_in_video(path,
                                                decoder_pipeline=default_pipeline,
                                                tag_pixel_diameter=tag_pixel_diameter,
                                                n_frames=n_frames,
                                                fps=fps,
                                                progress=progress)

        print(len(detections))
        # get the tracks                                           
        tracks = track_detections_dataframe(detections,
                                            tracker=tracker,
                                            confidence_filter_detections=confidence_filter_detections,
                                            confidence_filter_tracks=confidence_filter_tracks,
                                            coordinate_scale=coordinate_scale)
        
        display_tracking_results(path, frame_info, detections, tracks)

    
    except ValueError as err: #tritt auf, wenn Video leer ist. In diesem Fall: überspringe video
        print(err)
    except Exception as err:
        print(err)
        raise